# ***Word2Vec***

## Import Required Libraries

In [1]:
import pandas as pd
import re
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc

## Load Processed Data

In [4]:
df = pd.concat([
    pd.read_csv('/kaggle/input/train.csv'),
    pd.read_csv('/kaggle/input/valid.csv'),
    pd.read_csv('/kaggle/input/test.csv')
], ignore_index=False)

In [5]:
def convert(paragraph):
    paragraph = re.sub(r'(\s|^)[\d.,]*\d[\d.,]*(?=\s|$)', r'\1<NUM>', paragraph)
    paragraph = re.sub(r'(\s|^)(\d{1,2}h(\d{1,2})?)(?=\s|$)', r'\1<TIME>', paragraph)
    paragraph = re.sub(r'(\s|^)(\d{1,2}/\d{1,2}(/\d{2,4})?)(?=\s|$)', r'\1<DATE>', paragraph)
    return paragraph
paragraph = 'Tổng bí thư Nguyễn Phú Trọng phát biểu lúc 12h49 ngày 1/1/2990 . " Quốc hội khai mạc phiên họp thứ 2 ! "'
print(convert(paragraph))

Tổng bí thư Nguyễn Phú Trọng phát biểu lúc <TIME> ngày <DATE> . " Quốc hội khai mạc phiên họp thứ <NUM> ! "


In [6]:
def paragraph_to_tokens(paragraph: str) -> list:
    paragraph = convert(paragraph)
    parts = re.split(r'\s([.!?:])(?:\s+|$)', paragraph.strip())
    parts = [part for part in parts if part]
    sentences = []
    buffer = ''
    for part in parts:
        if part.strip() in '.!?"\'':
            buffer += ' ' + part.strip()
        else:
            if buffer:
                sentences.append(buffer)
            buffer = part.strip()
    if buffer:
        sentences.append(buffer.strip())
    tokens = [sentence.split() for sentence in sentences]
    return tokens

paragraph_test = 'Tổng bí thư Nguyễn Phú Trọng phát biểu lúc 12h49 ngày 1/1/2990 . " Quốc hội khai. mạc phiên họp thứ 2 ! " . '
print(paragraph_to_tokens(paragraph_test))

[['Tổng', 'bí', 'thư', 'Nguyễn', 'Phú', 'Trọng', 'phát', 'biểu', 'lúc', '<TIME>', 'ngày', '<DATE>', '.'], ['"', 'Quốc', 'hội', 'khai.', 'mạc', 'phiên', 'họp', 'thứ', '<NUM>', '!', '"', '.']]


In [7]:
all_sentences = []
tqdm.pandas()
for row in tqdm(df.itertuples(index=False)):
    all_sentences.extend(paragraph_to_tokens(row.summary))
    all_sentences.extend(paragraph_to_tokens(row.content))

438336it [04:44, 1539.68it/s]


In [8]:
len(all_sentences)

9312587

In [9]:
del df
gc.collect()

0

In [10]:
all_sentences = [sentence for sentence in all_sentences if len(sentence) >= 5]

In [11]:
len(all_sentences)

8160990

## Train

In [12]:
w2v_model = Word2Vec(
    sentences=all_sentences,
    vector_size=300,
    window=5,
    min_count=10,
    sg=1,
    workers=4,
    epochs=10
)

In [13]:
w2v_model.wv.most_similar('xe_máy')

[('xe_gắn_máy', 0.7692482471466064),
 ('ôtô', 0.7426812052726746),
 ('xe', 0.6776206493377686),
 ('xe_đạp', 0.6707013845443726),
 ('ô_tô', 0.669423520565033),
 ('xe_tải', 0.6261183023452759),
 ('mô_tô', 0.6144086718559265),
 ('môtô', 0.6059780716896057),
 ('xe_con', 0.5818155407905579),
 ('xe_hơi', 0.5446165800094604)]

In [14]:
w2v_model.wv.doesnt_match(['trường', 'giáo_viên', 'bệnh_viện', 'học_sinh'])

'bệnh_viện'

## Save model

In [15]:
w2v_model.save('word2vec_skipgram.model')